# Clasificador de Imágenes Simple

Este cuaderno te muestra cómo clasificar imágenes utilizando una red neuronal preentrenada.

**Lo que aprenderás:**
- Cómo cargar y usar un modelo preentrenado
- Preprocesamiento de imágenes
- Realizar predicciones en imágenes
- Comprender los puntajes de confianza

**Caso de uso:** Identificar objetos en imágenes (como "gato", "perro", "coche", etc.)

---


## Paso 1: Importar las bibliotecas necesarias

Vamos a importar las herramientas que necesitamos. ¡No te preocupes si aún no entiendes todas!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Paso 2: Cargar el modelo preentrenado

Usaremos **MobileNetV2**, una red neuronal ya entrenada con millones de imágenes.

Esto se llama **Aprendizaje por Transferencia**: ¡usar un modelo que alguien más entrenó!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Paso 3: Funciones Auxiliares

Vamos a crear funciones para cargar y preparar imágenes para nuestro modelo.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Paso 4: Prueba con imágenes de muestra

¡Vamos a intentar clasificar algunas imágenes de internet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Clasifica Cada Imagen


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Paso 5: ¡Prueba con tus propias imágenes!

Reemplaza la URL a continuación con cualquier URL de imagen que desees clasificar.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 ¿Qué acaba de pasar?

1. **Cargamos un modelo preentrenado** - MobileNetV2 fue entrenado con millones de imágenes.  
2. **Preprocesamos las imágenes** - Las redimensionamos y las formateamos para el modelo.  
3. **El modelo hizo predicciones** - Generó probabilidades para 1000 clases de objetos.  
4. **Decodificamos los resultados** - Convertimos los números en etiquetas comprensibles para humanos.  

### Entendiendo los niveles de confianza

- **90-100%**: Muy confiado (casi seguro que es correcto).  
- **70-90%**: Confiado (probablemente correcto).  
- **50-70%**: Algo confiado (podría ser correcto).  
- **Menos del 50%**: No muy confiado (incierto).  

### ¿Por qué podrían estar equivocadas las predicciones?

- **Ángulo o iluminación inusual** - El modelo fue entrenado con fotos típicas.  
- **Múltiples objetos** - El modelo espera un objeto principal.  
- **Objetos raros** - El modelo solo reconoce 1000 categorías.  
- **Imagen de baja calidad** - Las imágenes borrosas o pixeladas son más difíciles de interpretar.  

---


## 🚀 Próximos pasos

1. **Prueba con diferentes imágenes:**
   - Encuentra imágenes en [Unsplash](https://unsplash.com)
   - Haz clic derecho → "Copiar dirección de imagen" para obtener la URL

2. **Experimenta:**
   - ¿Qué sucede con arte abstracto?
   - ¿Puede reconocer objetos desde diferentes ángulos?
   - ¿Cómo maneja múltiples objetos?

3. **Aprende más:**
   - Explora las [lecciones de Visión por Computadora](../lessons/4-ComputerVision/README.md)
   - Aprende a entrenar tu propio clasificador de imágenes
   - Entiende cómo funcionan las CNNs (Redes Neuronales Convolucionales)

---

## 🎉 ¡Felicidades!

¡Acabas de construir un clasificador de imágenes utilizando una red neuronal de última generación!

Esta misma técnica impulsa:
- Google Photos (organización de tus fotos)
- Autos autónomos (reconocimiento de objetos)
- Diagnóstico médico (análisis de radiografías)
- Control de calidad (detección de defectos)

¡Sigue explorando y aprendiendo! 🚀



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
